Tranformation Phase

In [1]:
# Load Data
import pandas as pd

raw_df = pd.read_csv('data/raw_data.csv')
inc_df = pd.read_csv('data/incremental_data.csv')

1. Cleaning: Handle Missing Values and Remove Duplicates

Before any analysis, it's important to ensure data quality by handling missing values and removing duplicate records.

In [6]:
# Show missing values before cleaning
print("Raw Data missing values:\n", raw_df.isnull().sum())
print("Incremental Data missing values:\n", inc_df.isnull().sum())

# Show duplicates before cleaning
print("Raw Data duplicate rows:", raw_df.duplicated().sum())
print("Incremental Data duplicate rows:", inc_df.duplicated().sum())

Raw Data missing values:
 order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64
Incremental Data missing values:
 order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64
Raw Data duplicate rows: 0
Incremental Data duplicate rows: 0


 Cleaning Steps

- Fill missing `customer_name` and `region` with "Unknown".
- Fill missing `quantity` and `unit_price` with their respective median values.
- Drop rows with missing `order_date` (since dates are important for analysis).
- Remove duplicate rows.



In [7]:
# Fill missing 'customer_name' and 'region'
raw_df['customer_name'] = raw_df['customer_name'].fillna("Unknown")
raw_df['region'] = raw_df['region'].fillna("Unknown")
inc_df['customer_name'] = inc_df['customer_name'].fillna("Unknown")
inc_df['region'] = inc_df['region'].fillna("Unknown")

# Fill missing numeric values with the median
raw_df['quantity'] = raw_df['quantity'].fillna(raw_df['quantity'].median())
raw_df['unit_price'] = raw_df['unit_price'].fillna(raw_df['unit_price'].median())
inc_df['quantity'] = inc_df['quantity'].fillna(inc_df['quantity'].median())
inc_df['unit_price'] = inc_df['unit_price'].fillna(inc_df['unit_price'].median())

# Drop rows where 'order_date' is missing
raw_df = raw_df.dropna(subset=['order_date'])
inc_df = inc_df.dropna(subset=['order_date'])

# Remove duplicate rows
raw_df = raw_df.drop_duplicates()
inc_df = inc_df.drop_duplicates()

# Show missing values and shape after cleaning
print("Raw Data missing values after cleaning:\n", raw_df.isnull().sum())
print("Incremental Data missing values after cleaning:\n", inc_df.isnull().sum())
print("Raw Data shape after cleaning:", raw_df.shape)
print("Incremental Data shape after cleaning:", inc_df.shape)

Raw Data missing values after cleaning:
 order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64
Incremental Data missing values after cleaning:
 order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64
Raw Data shape after cleaning: (98, 7)
Incremental Data shape after cleaning: (10, 7)


2. Enrichment: Add total_price

To enrich the datasets, we calculate a new column called `total_price` as the product of `quantity` and `unit_price`. This provides immediate insight into the value of each order.

Let's view the data before and after the enrichment.

In [8]:
# Before enrichment
print(raw_df[['quantity', 'unit_price']].head())
print(inc_df[['quantity', 'unit_price']].head())

# Add total_price column
raw_df['total_price'] = raw_df['quantity'] * raw_df['unit_price']
inc_df['total_price'] = inc_df['quantity'] * inc_df['unit_price']

# After enrichment
print(raw_df[['quantity', 'unit_price', 'total_price']].head())
print(inc_df[['quantity', 'unit_price', 'total_price']].head())

   quantity  unit_price
0       2.0       500.0
1       2.0       500.0
2       2.0       250.0
3       2.0       750.0
4       3.0       500.0
   quantity  unit_price
0       1.5       900.0
1       1.0       300.0
2       1.0       600.0
3       1.5       300.0
4       2.0       600.0
   quantity  unit_price  total_price
0       2.0       500.0       1000.0
1       2.0       500.0       1000.0
2       2.0       250.0        500.0
3       2.0       750.0       1500.0
4       3.0       500.0       1500.0
   quantity  unit_price  total_price
0       1.5       900.0       1350.0
1       1.0       300.0        300.0
2       1.0       600.0        600.0
3       1.5       300.0        450.0
4       2.0       600.0       1200.0


3. Structural: Convert Dates and Ensure Data Types

It's important to have correct data types for analysis. Converting the `order_date` column to datetime and ensure `quantity` and `unit_price` are numeric.


In [9]:
# Before conversion
print(raw_df.dtypes)
print(inc_df.dtypes)

# Conversion
raw_df['order_date'] = pd.to_datetime(raw_df['order_date'], errors='coerce')
inc_df['order_date'] = pd.to_datetime(inc_df['order_date'], errors='coerce')
raw_df['quantity'] = pd.to_numeric(raw_df['quantity'], errors='coerce')
raw_df['unit_price'] = pd.to_numeric(raw_df['unit_price'], errors='coerce')
inc_df['quantity'] = pd.to_numeric(inc_df['quantity'], errors='coerce')
inc_df['unit_price'] = pd.to_numeric(inc_df['unit_price'], errors='coerce')

# After conversion
print(raw_df.dtypes)
print(inc_df.dtypes)

order_id                  int64
customer_name            object
product                  object
quantity                float64
unit_price              float64
order_date       datetime64[ns]
region                   object
total_price             float64
dtype: object
order_id                  int64
customer_name            object
product                  object
quantity                float64
unit_price              float64
order_date       datetime64[ns]
region                   object
total_price             float64
dtype: object
order_id                  int64
customer_name            object
product                  object
quantity                float64
unit_price              float64
order_date       datetime64[ns]
region                   object
total_price             float64
dtype: object
order_id                  int64
customer_name            object
product                  object
quantity                float64
unit_price              float64
order_date       datetime64[ns

3. Filtering: Drop Irrelevant Columns or Rows

If required, filtering out columns or rows that are not relevant. 

*(No columns dropped in this transformation.)*

4. Categorization: Create Customer Tiers

Creating a new column called `customer_tier` that categorizes customers based on their `total_price`:
- 'Bronze' for total_price < 500
- 'Silver' for total_price between 500 and 1000
- 'Gold' for total_price >= 1000


In [10]:
# Before
print(raw_df[['customer_name', 'total_price']].head())

# Categorization function
def tier(price):
    if price < 500:
        return 'Bronze'
    elif price < 1000:
        return 'Silver'
    else:
        return 'Gold'

raw_df['customer_tier'] = raw_df['total_price'].apply(tier)
inc_df['customer_tier'] = inc_df['total_price'].apply(tier)

# After
print(raw_df[['customer_name', 'total_price', 'customer_tier']].head())
print(inc_df[['customer_name', 'total_price', 'customer_tier']].head())

  customer_name  total_price
0         Diana       1000.0
1           Eve       1000.0
2       Charlie        500.0
3           Eve       1500.0
4           Eve       1500.0
  customer_name  total_price customer_tier
0         Diana       1000.0          Gold
1           Eve       1000.0          Gold
2       Charlie        500.0        Silver
3           Eve       1500.0          Gold
4           Eve       1500.0          Gold
  customer_name  total_price customer_tier
0         Alice       1350.0          Gold
1       Unknown        300.0        Bronze
2       Unknown        600.0        Silver
3       Unknown        450.0        Bronze
4         Heidi       1200.0          Gold


Save Transformed Datasets

Save the cleaned and enriched datasets to the `transformed/` folder as required:
- `transformed_full.csv` for the full dataset
- `transformed_incremental.csv` for the incremental dataset

In [11]:
raw_df.to_csv('transformed/transformed_full.csv', index=False)
inc_df.to_csv('transformed/transformed_incremental.csv', index=False)

Summary

- Cleaning: Missing values handled, duplicates removed
- Enrichment: Added total_price
- Structural: Converted dates and ensured numeric types
- Filtering: (No columns dropped)
- Categorization: Added customer_tier
- Transformed files saved for the next ETL step.